# Week09 — 추론 최적화 × FastAPI × 로드테스트 실습
**구성:** 이론(60') + 실습(40') + 과제

이 노트북은 다음을 단계별로 실습할 수 있게 구성되어 있습니다.

1. 환경 변수(.env) 템플릿 생성 및 로딩
2. 미니 챗봇 API(FastAPI) — 캐시/레이트리밋/스트리밍
3. (기본) Mock LLM / (옵션) OpenAI 연동 자리
4. 부하 테스트 스크립트(Locust, k6) 자동 생성
5. Postman/Insomnia 컬렉션 자동 생성
6. 캐시 키 실습, 레이트리밋 토큰버킷 실습
7. 실행/배포/측정 가이드 및 과제 안내


## 0) 사전 준비 파일 자동 생성
이 셀을 실행하면 **같은 폴더**에 아래 파일들이 생성됩니다.
- `.env.sample` — 환경 변수 템플릿
- `requirements_week09.txt` — 권장 패키지 목록
- `locustfile.py` — Locust 부하 테스트 스크립트
- `k6-loadtest.js` — k6 부하 테스트 스크립트
- `week09_postman_collection.json` — Postman 컬렉션
- `week09_insomnia_export.json` — Insomnia 워크스페이스(뼈대)


In [7]:
from pathlib import Path
from textwrap import dedent
import json, uuid, datetime

# 경로(노트북과 같은 폴더에 생성)
ENV_SAMPLE = Path(".env.sample")
REQS = Path("requirements_week09.txt")
LOCUST = Path("locustfile.py")
K6 = Path("k6-loadtest.js")
POSTMAN = Path("week09_postman_collection.json")
INSOMNIA = Path("week09_insomnia_export.json")

# 1) .env.sample
ENV_SAMPLE.write_text(dedent("""
# === Week09 Lab .env (샘플) ===
OPENAI_API_KEY="sk-~~~"
LANGFUSE_PUBLIC_KEY="pk-lf-~~~"
LANGFUSE_SECRET_KEY="sk-lf-~~~"
LANGFUSE_HOST="https://cloud.langfuse.com"
PINECONE_API_KEY="pcsk_~~~"

# 선택: Redis (로컬 Docker 기본값)
REDIS_URL="redis://localhost:6379/0"

# 모델/서비스 설정
MODEL_PROVIDER="mock"  # "mock" | "openai"
MODEL_NAME="gpt-4o-mini"  # openai 사용 시
RESPONSE_TTL_SECONDS=300
RATE_LIMIT_RPS=5
RATE_LIMIT_BURST=10
"""), encoding="utf-8")

# 2) requirements
REQS.write_text(dedent("""
fastapi>=0.115
uvicorn>=0.30
pydantic>=2.7
python-dotenv>=1.0
httpx>=0.27
redis>=5.0
orjson>=3.10
# openai>=1.52  # 필요 시 사용
# sse-starlette>=2.1  # (선택) SSE 라이브러리, 본 실습은 수동 스트리밍로 처리
"""), encoding="utf-8")

# 3) Locust
LOCUST.write_text(dedent("""
from locust import HttpUser, task, between
import json, random, string

def rand_prompt(n=12):
    return ''.join(random.choices(string.ascii_letters + ' ', k=n))

class ChatUser(HttpUser):
    wait_time = between(0.5, 1.5)
    @task(3)
    def chat(self):
        payload = {"userId":"locust", "message": f"Hello {rand_prompt()}"}
        self.client.post("/chat", json=payload)
    @task(1)
    def stream(self):
        payload = {"userId":"locust", "message": f"Stream {rand_prompt()}"}
        self.client.post("/chat/stream", json=payload)
"""), encoding="utf-8")

# 4) k6
K6.write_text(dedent("""
// k6 run k6-loadtest.js --vus 20 --duration 60s
import http from 'k6/http';
import { check, sleep } from 'k6';

export default function () {
  let payload = JSON.stringify({ userId: "k6", message: "hello from k6" });
  let headers = { 'Content-Type': 'application/json' };
  let res = http.post('http://localhost:8000/chat', payload, { headers });
  check(res, { 'status was 200': (r) => r.status === 200 });
  sleep(1);
}
"""), encoding="utf-8")

# 5) Postman 컬렉션
collection = {
  "info": {
    "name": "Week09 Mini Chatbot API",
    "_postman_id": str(uuid.uuid4()),
    "schema": "https://schema.getpostman.com/json/collection/v2.1.0/collection.json"
  },
  "item": [
    {
      "name": "Chat",
      "request": {
        "method": "POST",
        "header": [{"key": "Content-Type", "value": "application/json"}],
        "url": "http://localhost:8000/chat",
        "body": {"mode":"raw","raw":"{\n  \"userId\": \"demo\",\n  \"message\": \"Hello!\",\n  \"params\": {}\n}"}
      }
    },
    {
      "name": "Chat Stream",
      "request": {
        "method": "POST",
        "header": [{"key": "Content-Type", "value": "application/json"}],
        "url": "http://localhost:8000/chat/stream",
        "body": {"mode":"raw","raw":"{\n  \"userId\": \"demo\",\n  \"message\": \"Stream please\",\n  \"params\": {}\n}"}
      }
    }
  ]
}
POSTMAN.write_text(json.dumps(collection, indent=2), encoding="utf-8")

# 6) Insomnia (간단 워크스페이스 골격)
insomnia = {
  "_type": "export",
  "__export_format": 4,
  "__export_date": datetime.datetime.now().isoformat(),
  "__export_source": "week09-generator",
  "resources": [{
    "_id": "wrk_" + uuid.uuid4().hex,
    "_type": "workspace",
    "name": "Week09 Mini Chatbot API",
    "scope": "collection"
  }]
}
INSOMNIA.write_text(json.dumps(insomnia, indent=2), encoding="utf-8")

print("Artifacts written:")
print("-", ENV_SAMPLE)
print("-", REQS)
print("-", LOCUST)
print("-", K6)
print("-", POSTMAN)
print("-", INSOMNIA)


Artifacts written:
- .env.sample
- requirements_week09.txt
- locustfile.py
- k6-loadtest.js
- week09_postman_collection.json
- week09_insomnia_export.json


## 1) 환경 변수 로딩 (.env)
> 수업용으로 `.env`가 없으면 `.env.sample` 내용을 메모리에 로드합니다. 실제 프로젝트에서는 `python-dotenv`를 사용하세요.


In [8]:
import os
from pathlib import Path

if not Path('.env').exists() and Path('.env.sample').exists():
    for line in Path('.env.sample').read_text(encoding='utf-8').splitlines():
        line = line.strip()
        if not line or line.startswith('#'):
            continue
        if '=' in line:
            k, v = line.split('=', 1)
            os.environ[k.strip()] = v.strip().strip('"').strip("'")

CONFIG = {
    "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY", ""),
    "LANGFUSE_PUBLIC_KEY": os.getenv("LANGFUSE_PUBLIC_KEY", ""),
    "LANGFUSE_SECRET_KEY": os.getenv("LANGFUSE_SECRET_KEY", ""),
    "LANGFUSE_HOST": os.getenv("LANGFUSE_HOST", ""),
    "PINECONE_API_KEY": os.getenv("PINECONE_API_KEY", ""),
    "REDIS_URL": os.getenv("REDIS_URL", "redis://localhost:6379/0"),
    "MODEL_PROVIDER": os.getenv("MODEL_PROVIDER", "mock"),
    "MODEL_NAME": os.getenv("MODEL_NAME", "gpt-4o-mini"),
    "RESPONSE_TTL_SECONDS": int(os.getenv("RESPONSE_TTL_SECONDS", "300")),
    "RATE_LIMIT_RPS": int(os.getenv("RATE_LIMIT_RPS", "5")),
    "RATE_LIMIT_BURST": int(os.getenv("RATE_LIMIT_BURST", "10")),
}
CONFIG


{'OPENAI_API_KEY': 'sk-~~~',
 'LANGFUSE_PUBLIC_KEY': 'pk-lf-~~~',
 'LANGFUSE_SECRET_KEY': 'sk-lf-~~~',
 'LANGFUSE_HOST': 'https://cloud.langfuse.com',
 'PINECONE_API_KEY': 'pcsk_~~~',
 'REDIS_URL': 'redis://localhost:6379/0',
 'MODEL_PROVIDER': 'mock"  # "mock" | "openai',
 'MODEL_NAME': 'gpt-4o-mini"  # openai 사용 시',
 'RESPONSE_TTL_SECONDS': 300,
 'RATE_LIMIT_RPS': 5,
 'RATE_LIMIT_BURST': 10}

## 2) 미니 챗봇 API 서버 코드 생성 (FastAPI)
- 엔드포인트:
  - `POST /chat` : 일반 응답(캐시/레이트리밋 적용)
  - `POST /chat/stream` : SSE 스타일 스트리밍 모사
- 구현 요소:
  - **응답 캐시**: `sha256(prompt+model+params)` 키 + TTL
  - **레이트 리밋**: 메모리 토큰버킷(실서비스는 Redis 권장)
  - **모의 LLM(MockLLM)**: OpenAI 없이도 동작

> 아래 셀은 `/mnt/data/week09_app` 폴더에 서버 소스파일을 생성합니다.


In [9]:
from pathlib import Path
from textwrap import dedent

APP_DIR = Path('week09_app')
APP_DIR.mkdir(exist_ok=True)

# utils_hash.py
(APP_DIR / 'utils_hash.py').write_text(dedent(r'''
import hashlib, json

def cache_key(payload: dict, model_name: str, schema_version: str="v1") -> str:
    blob = json.dumps({"payload": payload, "model": model_name, "schema": schema_version}, sort_keys=True)
    return hashlib.sha256(blob.encode("utf-8")).hexdigest()
'''), encoding='utf-8')

# rate_limit.py (토큰버킷)
(APP_DIR / 'rate_limit.py').write_text(dedent(r'''
import time
from collections import defaultdict

class TokenBucket:
    def __init__(self, rate_per_sec: int, burst: int):
        self.rate = rate_per_sec
        self.burst = burst
        self.tokens = defaultdict(lambda: burst)
        self.timestamps = defaultdict(lambda: time.time())

    def allow(self, key: str) -> bool:
        now = time.time()
        last = self.timestamps[key]
        delta = now - last
        refill = delta * self.rate
        self.tokens[key] = min(self.burst, self.tokens[key] + refill)
        self.timestamps[key] = now
        if self.tokens[key] >= 1:
            self.tokens[key] -= 1
            return True
        return False
'''), encoding='utf-8')

# cache_mem.py (TTL 캐시)
(APP_DIR / 'cache_mem.py').write_text(dedent(r'''
import time

class TTLCache:
    def __init__(self, ttl_seconds: int = 300):
        self.ttl = ttl_seconds
        self.store = {}

    def get(self, key: str):
        item = self.store.get(key)
        if not item: return None
        value, exp = item
        if exp < time.time():
            self.store.pop(key, None)
            return None
        return value

    def set(self, key: str, value):
        self.store[key] = (value, time.time() + self.ttl)
'''), encoding='utf-8')

# mock_llm.py (지연/스트리밍 모사)
(APP_DIR / 'mock_llm.py').write_text(dedent(r'''
import asyncio

class MockLLM:
    def __init__(self, delay_ms_per_token: int = 40):
        self.delay = delay_ms_per_token / 1000.0

    async def generate(self, prompt: str, max_tokens: int = 64):
        text = "[MOCK] " + (prompt[::-1])[:max_tokens]
        await asyncio.sleep(self.delay * max(1, min(len(text)//6, 10)))
        return text

    async def stream(self, prompt: str, max_tokens: int = 64):
        text = await self.generate(prompt, max_tokens)
        chunks = max(1, min(8, len(text)//max(1, len(text)//8)))
        step = max(1, len(text)//chunks)
        for i in range(0, len(text), step):
            await asyncio.sleep(self.delay)
            yield text[i:i+step]
'''), encoding='utf-8')

# main.py (FastAPI 앱)
(APP_DIR / 'main.py').write_text(dedent(r'''
from fastapi import FastAPI, HTTPException, Request
from fastapi.responses import StreamingResponse, JSONResponse
from pydantic import BaseModel, Field
import os, json

from utils_hash import cache_key
from rate_limit import TokenBucket
from cache_mem import TTLCache
from mock_llm import MockLLM

MODEL_NAME = os.getenv("MODEL_NAME", "gpt-4o-mini")
RESPONSE_TTL_SECONDS = int(os.getenv("RESPONSE_TTL_SECONDS", "300"))
RATE_LIMIT_RPS = int(os.getenv("RATE_LIMIT_RPS", "5"))
RATE_LIMIT_BURST = int(os.getenv("RATE_LIMIT_BURST", "10"))

app = FastAPI(title="Week09 Mini Chatbot API")

bucket = TokenBucket(rate_per_sec=RATE_LIMIT_RPS, burst=RATE_LIMIT_BURST)
cache = TTLCache(ttl_seconds=RESPONSE_TTL_SECONDS)
llm = MockLLM()

class ChatRequest(BaseModel):
    userId: str = Field(..., description="사용자 식별자")
    message: str = Field(..., description="프롬프트/메시지")
    params: dict = Field(default_factory=dict, description="샘플링 파라미터 등 선택값")

@app.post("/chat")
async def chat(req: ChatRequest, request: Request):
    client_ip = request.client.host if request.client else "unknown"
    rl_key = f"{client_ip}:{req.userId}"
    if not bucket.allow(rl_key):
        raise HTTPException(status_code=429, detail="Rate limit exceeded")

    key = cache_key({"message": req.message, "params": req.params}, MODEL_NAME)
    cached = cache.get(key)
    if cached:
        return JSONResponse({"cached": True, "model": MODEL_NAME, "output": cached})

    # (옵션) OpenAI 호출 자리 — 현재는 MockLLM 사용
    text = await llm.generate(req.message, max_tokens=128)
    cache.set(key, text)
    return {"cached": False, "model": MODEL_NAME, "output": text}

@app.post("/chat/stream")
async def chat_stream(req: ChatRequest, request: Request):
    client_ip = request.client.host if request.client else "unknown"
    rl_key = f"{client_ip}:{req.userId}"
    if not bucket.allow(rl_key):
        raise HTTPException(status_code=429, detail="Rate limit exceeded")

    async def streamer():
        first = True
        async for chunk in llm.stream(req.message, max_tokens=128):
            data = json.dumps({"delta": chunk, "done": False if first else None})
            first = False
            yield f"data: {data}\n\n"
        yield 'data: {"done": true}\n\n'

    headers = {"Content-Type": "text/event-stream"}
    return StreamingResponse(streamer(), headers=headers)
'''), encoding='utf-8')

print(f"App files written under: {APP_DIR}")
print("- utils_hash.py, rate_limit.py, cache_mem.py, mock_llm.py, main.py")


App files written under: week09_app
- utils_hash.py, rate_limit.py, cache_mem.py, mock_llm.py, main.py


## 3) 로컬 실행 방법
```bash
# 가상환경 & 설치
python -m venv .venv && source .venv/bin/activate   # Windows: .venv\Scripts\activate
pip install -r requirements_week09.txt

# (옵션) Redis 실행
docker run -d --name redis -p 6379:6379 redis:7-alpine

# .env 준비
cp .env.sample .env

# 서버 실행 (노트북에서 생성된 경로)
cd week09/week09_app
uvicorn main:app --host 0.0.0.0 --port 8000 --workers 1 --reload
```


## 4) 노트북에서 간단 요청 테스트 (옵션)
> 아래 코드는 로컬에서 8000 포트로 서버가 이미 떠 있다고 가정합니다.


In [11]:
import httpx, json, os

BASE = os.getenv("WEEK09_BASE_URL", "http://localhost:8000")
try:
    r = httpx.get(f"{BASE}/docs", timeout=3.0)
    print("Server /docs:", r.status_code)
except Exception as e:
    print("서버가 떠있지 않은 것 같습니다. 로컬에서 uvicorn을 먼저 실행하세요.\n", e)

payload = {"userId":"notebook","message":"안녕하세요 FastAPI","params":{}}
try:
    r = httpx.post(f"{BASE}/chat", json=payload, timeout=5.0)
    print("POST /chat:", r.status_code)
    if r.status_code == 200:
        print(json.dumps(r.json(), ensure_ascii=False, indent=2))
except Exception as e:
    print("요청 실패:", e)


Server /docs: 200
POST /chat: 200
{
  "cached": true,
  "model": "gpt-4o-mini",
  "output": "[MOCK] IPAtsaF 요세하녕안"
}


## 5) 부하 테스트 (Locust / k6)
**Locust**
```bash
locust -f locustfile.py --host http://localhost:8000
# 브라우저에서 http://localhost:8089 접속 → VU/Spawn rate 설정 → Start
```
**k6**
```bash
k6 run k6-loadtest.js --vus 20 --duration 60s
```
> 성능 지표는 p50/p95/p99, 에러율, 처리량(req/s) 중심으로 기록하세요.


## 6) 캐시 키 실습
동일 페이로드(+모델)이면 캐시가 히트되어 두 번째 요청은 즉시 응답됩니다.


In [12]:
from importlib import import_module
cache_mod = import_module('week09_app.utils_hash')

p1 = {"message":"Hello","params":{"temp":0.7}}
p2 = {"message":"Hello!","params":{"temp":0.7}}
print("Key1:", cache_mod.cache_key(p1, "gpt-4o-mini"))
print("Key2:", cache_mod.cache_key(p2, "gpt-4o-mini"))
print("Same? ->", cache_mod.cache_key(p1, "gpt-4o-mini") == cache_mod.cache_key(p2, "gpt-4o-mini"))


Key1: 2264763028d66f2f90cd6e24ee10ed2cbb870829869fc95749837ce3d5ee8042
Key2: 93eb519ddb7bb6369328a79451e1bb44929369f93c96c27e9cb5464cf01d0ddb
Same? -> False


## 7) 레이트 리밋 토큰버킷 개념 실습
동일 키(예: `client_ip:userId`)로 짧은 시간 안에 여러 번 호출하면 429가 발생합니다. 실서비스는 Redis/슬라이딩윈도우/분산 토큰버킷 권장.


In [13]:
from importlib import import_module
import time
rl_mod = import_module('week09_app.rate_limit')

bucket = rl_mod.TokenBucket(rate_per_sec=3, burst=3)
key = "demo"
allowed = []
for i in range(10):
    allowed.append(bucket.allow(key))
    time.sleep(0.1)

print("허용 여부:", allowed)
print("설명: 처음 3개는 버스트로 허용, 이후에는 초당 3개 속도로 리필됩니다.")


허용 여부: [True, True, True, False, True, False, False, True, False, False]
설명: 처음 3개는 버스트로 허용, 이후에는 초당 3개 속도로 리필됩니다.


## 8) 과제 가이드 (필수/가점)
### 필수 제출물
- **Mini Chatbot API(FastAPI)** — `/chat`, `/chat/stream`
- **부하 테스트 결과** — p50/p95/p99, 에러율, 처리량(req/s) 표/그래프 정리

### 가점 항목
- **모델서버 분리** (vLLM/TGI/Ollama) — FastAPI는 게이트웨이
- **캐시** — Redis 기반 응답 캐시 + 히트율 측정
- **레이트 리밋** — 사용자/토큰 단위 분산 레이트 리밋
- **스트리밍** — SSE vs WebSocket 비교 및 체감 지연 분석

### 제출 형태
- GitHub 저장소 링크 + README(구현/측정/분석) + 부하테스트 스크린샷
- Postman/Insomnia 컬렉션 파일 포함
